In [43]:
import vowpalwabbit
from analysis import analysis_data
import os.path as osp
import pandas as pd
from sklearn.model_selection import train_test_split

vw = vowpalwabbit.Workspace("--cb 4", quiet=True)

In [44]:
run_data_path = osp.join(osp.dirname(analysis_data.__file__), "vowpal_data.csv")
df = pd.read_csv(run_data_path)
df.columns = ["action", "cost", "probability", "consumption"]
df.head()

,action,cost,probability,consumption
0,-0.05,0.0000,0.058824,1.370667e+00
1,-0.02,0.0000,0.058824,1.018524e-07
2,0.01,0.0000,0.058824,1.928167e+00
3,0.30,0.0000,0.058824,5.158833e-01
4,0.05,3.6864,0.058824,8.346000e-01


In [45]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(["cost"], axis=1), df["cost"], test_size=0.1, random_state=42)
X_train

,action,probability,consumption
498,0.02,0.058824,0.286300
31050,-0.10,0.058824,1.098750
40671,-0.05,0.058824,1.202417
7375,0.15,0.058824,-0.198533
16062,-0.02,0.058824,1.751300
...,...,...,...
6265,0.20,0.058824,0.608533
11284,0.02,0.058824,0.588550
38158,0.05,0.058824,0.181417
860,0.15,0.058824,0.273750


In [46]:
train_df = pd.concat([X_train, y_train], axis=1)
train_df.head()

,action,probability,consumption,cost
498,0.02,0.058824,0.286300,11.989514
31050,-0.10,0.058824,1.098750,609.614764
40671,-0.05,0.058824,1.202417,236.181489
7375,0.15,0.058824,-0.198533,0.673949
16062,-0.02,0.058824,1.751300,37.247129


In [47]:
for i in train_df.index:
    action = train_df.loc[i, "action"]
    cost = train_df.loc[i, "cost"]
    probability = train_df.loc[i, "probability"]
    feature1 = train_df.loc[i, "consumption"]

    # Construct the example in the required vw format.
    learn_example = (
        str(action)
        + ":"
        + str(cost)
        + ":"
        + str(probability)
        + " | "
        + str(feature1)
    )

    # Here we do the actual learning.
    vw.learn(learn_example)

In [48]:
test_df = pd.concat([X_test, y_test], axis=1)
test_df.head()

,action,probability,consumption,cost
35375,0.07,0.058824,1.787567,374.560951
3161,0.15,0.058824,0.499567,37.385745
14165,-0.01,0.058824,0.951250,76.052204
34294,-0.15,0.058824,0.338950,190.276327
6312,-0.02,0.058824,-1.555000,1.138724


In [50]:
for j in test_df.index:
    feature1 = test_df.loc[j, "consumption"]

    test_example = "| " + str(feature1)

    choice = vw.predict(test_example)
    print(feature1, j, choice)

1.7875666666666663 35375 1
0.4995666666666671 3161 1
0.9512499999999992 14165 1
0.3389500299999992 34294 1
-1.5549999997456865 6312 1
0.695816666666667 6948 1
0.8025166666666672 26487 1
0.6793666666666662 18505 1
-2.3027333333333337 28751 1
0.2647499999999991 43401 1
-2.2398333333333333 28616 1
0.9934833333333334 28338 1
1.1120999986012772 15397 1
0.992716666666666 21379 1
-1.07685 32478 1
0.7689750000000005 36235 1
0.5040166666666671 25335 1
-2.0347166666666667 12283 1
-2.7198833333333328 24769 1
0.5555833333333328 43340 1
-0.8937333333333337 3308 1
0.2804500399999989 13912 1
-2.239583333333333 36784 1
0.17035 21496 1
1.0752666666666668 31788 1
2.8493833333333334 40554 1
0.3582999999999998 11753 1
-1.7851333297729493 12776 1
1.6233833333333327 17784 1
0.4556000000000001 4193 1
0.5642333333333328 36135 1
0.3710500000000001 28218 1
-1.068083333333333 37858 1
-1.2742833333333334 11925 1
0.1221666666666666 37224 1
0.5612166666666666 18494 1
0.2579249999999999 29308 1
0.6680833333333336 16

In [41]:
vw.save("cb.model")
del vw

vw = vowpalwabbit.Workspace("--cb 4 -i cb.model", quiet=True)
print(vw.predict("| a b"))

1
